## Getting params from existing PowerBI query

In [49]:
import urllib
import json

url = """https://api.supermetrics.com/enterprise/v2/query/data/powerbi?json={"a":"b"}"""
url_unquoted = urllib.parse.unquote(url)
s = urllib.parse.parse_qs(url_unquoted)
params = s["https://api.supermetrics.com/enterprise/v2/query/data/powerbi?json"][0]
params_d = json.loads(params)
params_d

{'a': 'b'}

## Downloading data

Example

In [52]:
import requests
import urllib
import os

# os.environ["SUPERMETRICS_API_KEY"] = 
SUPERMETRICS_API_KEY = os.environ["SUPERMETRICS_API_KEY"]

base_url = "https://api.supermetrics.com/enterprise/v2/query/data/json"
json_params = {
        "ds_id": "AW",
        "ds_accounts": ["1007802423"],
        "ds_user": "google@velux.com",
        "date_range_type": "last_year_inc",
        "fields": ["Date",
        "profile",
        "Campaignname",
        "Impressions",
        "Clicks",
        "Cost_eur",
        "SearchImpressionShare"],
        # "max_rows": 1000000,
        "max_rows": 3,
        "api_key": SUPERMETRICS_API_KEY
}
params = {"json": json.dumps(json_params)}
r = requests.get(base_url, params=params)
assert r.ok
response_json = r.json()
assert response_json["meta"]["status_code"] == "SUCCESS"
data = response_json["data"]
data

[['Date',
  'Account',
  'Campaign name',
  'Impressions',
  'Clicks',
  'Cost (EUR)',
  'Search impression share'],
 ['2020-01-01',
  'VELUXSCoE - FR',
  'FR : 00 : Brand VELUX (Exact)',
  812,
  327,
  55.88,
  0.9409],
 ['2020-01-01',
  'VELUXSCoE - FR',
  'FR : 00 : Brand VELUX (PHRASE)',
  2406,
  266,
  198.39,
  0.3636],
 ['2020-01-01',
  'VELUXSCoE - FR',
  'FR : 01 : Brand VELUX Flat Roof Window - Coupoles',
  1,
  0,
  0,
  0.5]]

In [53]:
import pandas as pd
df = pd.DataFrame(data[1:], columns=data[0])
df

,Date,Account,Campaign name,Impressions,Clicks,Cost (EUR),Search impression share
0,2020-01-01,VELUXSCoE - FR,FR : 00 : Brand VELUX (Exact),812,327,55.88,0.9409
1,2020-01-01,VELUXSCoE - FR,FR : 00 : Brand VELUX (PHRASE),2406,266,198.39,0.3636
2,2020-01-01,VELUXSCoE - FR,FR : 01 : Brand VELUX Flat Roof Window - Coupoles,1,0,0.00,0.5000


## Building a flow

In [56]:
import prefect
from prefect import task, Flow
from prefect.storage import GitHub
from prefect.run_configs import DockerRun
from prefect.tasks.secrets import PrefectSecret

import requests
import urllib
import json


### Local run setup
# os.environ["SUPERMETRICS_API_KEY"] = 
os.environ["SUPERMETRICS_USER"] = "google@velux.com"
prefect.context.setdefault("secrets", {})
prefect.context.secrets["SUPERMETRICS_API_KEY"] = os.environ["SUPERMETRICS_API_KEY"]
prefect.context.secrets["SUPERMETRICS_USER"] = os.environ["SUPERMETRICS_USER"]
### 

### Production setup
# STORAGE = GitHub(
#     repo="dyvenia/platform",
#     path="prefect/flows/supermetrics.py",
#     access_token_secret="github_token",  # name of the Prefect secret with the GitHub token
# )

# RUN_CONFIG = DockerRun(
#     image="docker.pkg.github.com/dyvenia/platform/dyvenia:latest",
#     env={"SOME_VAR": "value"},
#     labels=["velux"],
# )
### 

@task
def supermetrics_to_json(user, api_key):

    logger = prefect.context.get("logger")
    logger.info("Downloading !")

    API_ENDPOINT = "https://api.supermetrics.com/enterprise/v2/query/data/json"
    json_params = {
            "ds_id": "AW",
            "ds_accounts": ["1007802423"],
            "ds_user": user,
            "date_range_type": "last_year_inc",
            "fields": ["Date",
            "profile",
            "Campaignname",
            "Impressions",
            "Clicks",
            "Cost_eur",
            "SearchImpressionShare"],
            # "max_rows": 1000000,
            "max_rows": 3,
            "api_key": api_key
    }
    params = {"json": json.dumps(json_params)}
    response = requests.get(API_ENDPOINT, params=params)
    assert response.ok

    response_json = response.json()
    assert response_json["meta"]["status_code"] == "SUCCESS"

    data = response_json["data"]
    return data
    
@task
def to_df(data):
    import pandas as pd
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

@task
def show_df(df):
    logger = prefect.context.get("logger")
    logger.info(f"Data:\n\n{df.to_markdown()}\n\n")


with Flow("Supermetrics") as flow: # , storage=STORAGE, run_config=RUN_CONFIG) as flow:
    SUPERMETRICS_API_KEY = PrefectSecret("SUPERMETRICS_API_KEY")
    SUPERMETRICS_USER = PrefectSecret("SUPERMETRICS_USER")
    data = supermetrics_to_json(user=SUPERMETRICS_USER, api_key=SUPERMETRICS_API_KEY)
    df = to_df(data)
    pretty_print = show_df(df)

# flow.register(project_name="velux")

# if __name__ == "__main__":
#     flow.run()
flow.run()

[2021-04-14 10:36:42+0000] INFO - prefect.FlowRunner | Beginning Flow run for 'Supermetrics'
[2021-04-14 10:36:42+0000] INFO - prefect.TaskRunner | Task 'SUPERMETRICS_API_KEY': Starting task run...
[2021-04-14 10:36:42+0000] INFO - prefect.TaskRunner | Task 'SUPERMETRICS_API_KEY': Finished task run for task with final state: 'Success'
[2021-04-14 10:36:43+0000] INFO - prefect.TaskRunner | Task 'SUPERMETRICS_USER': Starting task run...
[2021-04-14 10:36:43+0000] INFO - prefect.TaskRunner | Task 'SUPERMETRICS_USER': Finished task run for task with final state: 'Success'
[2021-04-14 10:36:43+0000] INFO - prefect.TaskRunner | Task 'supermetrics_to_json': Starting task run...
[2021-04-14 10:36:43+0000] INFO - prefect.supermetrics_to_json | Downloading !
[2021-04-14 10:36:44+0000] INFO - prefect.TaskRunner | Task 'supermetrics_to_json': Finished task run for task with final state: 'Success'
[2021-04-14 10:36:44+0000] INFO - prefect.TaskRunner | Task 'to_df': Starting task run...
[2021-04-14 

<Success: "All reference tasks succeeded.">